# Rewrite Reuse Refactor
## Reproducing Unit Tests using refactored ODOG formulation
This Notebook introduces a new way of formulating the normalization step of the ODOG family of Models, and shows that this formulation is numerically equivalent to the original paper implementation, to establish continuity

In [ ]:
# Third party libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy

# Import local module
import multyscale

In [ ]:
# %% Load example stimulus
stimulus = np.asarray(Image.open("example_stimulus.png").convert("L"))

# %% Parameters of image
# visual extent, same convention as pyplot:
visextent = (-16, 16, -16, 16)

In [ ]:
model = multyscale.models.ODOG_RHS2007(shape=stimulus.shape, visextent=visextent)

In [ ]:
# Frontend filterbank of ODOG implementation by Robinson et al. (2007)
O, S = model.bank.filters.shape[:2]

In [ ]:
# Filter the example stimulus
filters_output = model.bank.apply(stimulus)

In [ ]:
# Weight individual filter(outputs) according to spatial size (frequency)
filters_output = model.weight_outputs(filters_output)

This preamble defines a filterbank output which is not already normalized by any function. The Traditional ODOG normalization function is multyscale/models.py ODOG, shown below

In [ ]:
normalized_outputs = model.normalize_outputs(filters_output)

We're comparing this to a new implementation which replaces the math of the original function by a more general form.  
The traditional formulation for the normalization schema of ODOG can be formulated as:
$$ F' = \frac{f_{o',s',x,y}}{\sqrt{\frac{1}{XY}\sum_{y=1}^{Y} \sum_{x=1}^{X}(\sum_{o=1}^{O}\sum_{s=1}^{S} {w_{o',s',o,s} f_{o,s,x,y})^2}}}$$
where $w_{o', s', o, s} =   \begin{cases} 
      1 & o = o'  \\
      0 & else
   \end{cases} 
$

In [ ]:
normalizers = model.normalizers(filters_output)

RMS = model.normalizers_to_RMS(normalizers)

normed2 = np.ndarray(filters_output.shape)
for o,s in np.ndindex(filters_output.shape[:2]):
    normed2[o,s] = filters_output[o,s] / RMS[o,s]

assert(np.allclose(normalized_outputs,normed2))

#### Can also divide by "images"

In [ ]:
i_RMS = np.tile(RMS.reshape(6,7,1,1),(1,1,1024,1024))

norm_i_outputs = np.ndarray(filters_output.shape)
for o, s in np.ndindex(filters_output.shape[:2]):
    f = filters_output[o,s,...]
    n = i_RMS[o,s]
    norm_i_outputs[o, s] = f / n

assert(np.allclose(normalized_outputs, norm_i_outputs))

This is equivalent to an equation that implements the averaging in the denominator as a linear filter, that scales the filter aggregate produced by the sum over scales/orientations. This changes the dimensionality of the denominator, but not actually the result of the division, since image by image division is executed pixel-wise

$$ODOG: \frac{f_{o^*,s^*}}{\sqrt{G_{x,y}*(\sum_{o=1}^{O}\sum_{s=1}^{S} {w_{o',s',o,s} f_{o,s,x,y})^2}}}$$
$w_{o,s}$ same as above,  
$G_{x,y} = \frac{1}{XY} = \frac{1}{1024^2}$ everywhere.

In [ ]:
filter = np.ones((1024,1024)) / 1024**2

iRMS2 = np.ones(filters_output.shape)
#for o,s in np.ndindex(filters_output.shape[:2]):
o=3
s=4
# calculate RMS using the filter
norm = normalizers[o,s].copy()
norm = norm ** 2
#mean = multyscale.filters.apply(norm, filter, padval=norm.mean())
#mean = norm.mean()
mean = scipy.signal.convolve2d(norm, filter, mode="same", boundary="symm")
iRMS2[o,s] = iRMS2[o,s]*np.sqrt(mean)

plt.imshow(mean)
mean.shape
#assert(np.allclose(iRMS2, iRMS))

By renaming the remaining sum in the denominator, we can find an abstract formulation of the RMS-Norm as a sequence of linear filters

 $$\frac{f_{o^*,s^*}}{\sqrt{G_{x,y}*(\sum_{o=1}^{O}\sum_{s=1}^{S} {w_{o',s',o,s} f_{o,s,x,y})^2}}}=\frac{f_{o^*,s^*}}{\sqrt{G * N^2}} = \frac{f_{o^*,s^*}}{\sqrt{G * (W \cdot f)^2}}$$
 
 Which helps us develop terminology. The entire fraction is called the divisive norm on the filterbank-output $f$.  
 $G * N^2$ is the (local) energy, calculated as the convolution of $G$ the spatial weighting, and $N^2$ the Normcoefficient image squared.  
The Normcoefficient Image can be calculated in turn by computing the Normpool $W$ dotproduct against the same $f$ that we are normalizing.
   
An implementation of this formulation can be found here:

In [ ]:
def divisive_norm(filter_outputs, o, s):
    G = spatial_weighting(filter_outputs.shape[-2:])
    N = normcoeff(filter_outputs, o, s)**2
    
    z = scipy.signal.convolve(G, N, mode="valid")
    normed_filter = filter_outputs[o,s,:] / np.sqrt(z)
    return normed_filter

def normcoeff(filter_outputs, o_0, s_0): 
    w = normpool(filter_outputs.shape[0],filter_outputs.shape[1], o_0, s_0)
    coeffs = np.tensordot(filter_outputs, w, axes=([0, 1], [0, 1]))
    return coeffs

def normpool(O, S, o_0, s_0): # w_{o,s}
    ODOG = True
    LODOG, FLODOG = False, False
    w = np.ones(shape=(O,S))
    for o in range(O):
        for s in range(S):
            if ODOG or LODOG:
                if o==o_0:
                    w[o,s] = 1/S
                else:
                    w[o,s] = 0

            if FLODOG:
                if o==o_0:
                    w[o,s] = 1 #gaussian(s-s_0)
                else:
                    w[o,s] = 0
    return w

def spatial_weighting(filter_shape):
    ODOG = True
    LODOG, FLODOG = False, False
    if ODOG:
        G = np.ones(filter_shape) / np.prod(filter_shape)
        
    if LODOG:
        G = 1 #gaussian(filter_shape, sigma)

    if FLODOG:
        G = 1 #gaussian(filter_shape, sigma=k*s)
    return G

Our goal is now to establish numerical equivalence between the traditional formulation and the result of the newly reformatted and mathematically motivated code

The original implementation is already tested against the matlab implementation of RHS_2007, so we know it is sound.
  
We will use this notebook to figure out: 1) if all of the units of our new implementation act as expected and 2) if the integration of those units is again comparable to RHS_2007

Note that in the module, all of the below functions will appear in multyscale/test, separated into fixtures and tests.

In [ ]:
def filtershape():
    return (1024,1024)

def test_unit_spatial_weighting(filter_shape):
    G = spatial_weighting(filter_shape)
    assert np.allclose(G, 1/np.prod(filter_shape))
    
test_unit_spatial_weighting(filtershape())

def filteroutput():
    return np.random.random(size=(6,7,1024,1024))


def test_unit_normcoeff(filter_outputs):
    # use sum over scales formulation instead of linear filter
    if input("Run full 6x7 analysis? [y/n] ")=="y":
        N = np.zeros(shape=(6,7,1024,1024))
        for o, s in np.ndindex(filter_outputs.shape[:2]):
            for x,y in np.ndindex(filter_outputs.shape[2:]):
                N[o, s, x, y] = np.sum(filter_outputs[o,:,x,y]) / filter_outputs.shape[1]
            assert np.allclose(normcoeff(filter_outputs, o, s), N[o, s])

            print(f"dimension {o},{s} is sound")
    else:
        print("only doing 5 dimensions chosen at random")
        N = np.zeros(shape=(6,7,1024,1024))
        for i in range(5):
            o, s = np.random.randint(0,6), np.random.randint(0,7)
            for x,y in np.ndindex(filter_outputs.shape[2:]):
                N[o, s, x, y] = np.sum(filter_outputs[o,:,x,y]) / filter_outputs.shape[1] # Does W.f properly produce sum_o,s[w_o,s . f_o,s]
            assert np.allclose(normcoeff(filter_outputs, o, s), N[o, s])

            print(f"dimension {o},{s} is sound")
        

print("unit test normcoeff")
test_unit_normcoeff(filteroutput())
print("unit test divisive")

def test_unit_divisive(filter_outputs):
    for i in range(5):
        o, s = np.random.randint(0,6), np.random.randint(0,7)
        G = spatial_weighting(filter_outputs.shape[-2:])
        N = normcoeff(filter_outputs, o, s)**2
        z = np.mean(N)  # does N*G properly calculate mean(N)?
        normed_filter = filter_outputs[o,s,:] / np.sqrt(z)
        assert np.allclose(divisive_norm(filter_outputs, o, s), normed_filter)

        print(f"dimension {o},{s} is sound")
test_unit_divisive(filteroutput())

So as one can see, all of the steps agree with the mathematical formulation of ODOG. The next step will be, to analyze whether the new formulation agrees not only with the math, but also with the already verified original implementation that is present in the current codebase.

In [ ]:
import scipy

def normalize_multiple_outputs(filters_output):

    normalized_outputs = np.ones(shape=filters_output.shape)
    for o, s in np.ndindex(filters_output.shape[:2]):
        normalized_outputs[o,s] = divisive_norm(filters_output, o,s)

    return normalized_outputs

def test_integration_odog_reform(output_odog_matlab, stimulus):
    model = multyscale.models.ODOG_RHS2007(stimulus.shape, visextent)
    output2 = model.apply(stimulus)
    model.normalize_outputs = normalize_multiple_outputs
    output = model.apply(stimulus)
    assert np.allclose(output, output2)
    #assert np.allclose(output, output_odog_matlab)
    
test_integration_odog_reform("", stimulus)

This code takes a while to complete, and has a very dissatisfying ending. The formulation right now cannot reproduce traditional output